<a href="https://colab.research.google.com/github/sreyeshkonduru/sreyesh_INFO5731_Fall2024/blob/main/KONDURU_SREYESH_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [1]:
import requests
import pandas as pd
import time
import subprocess
import platform

queries = ["machine learning", "data science", "artificial intelligence", "information extraction"]
api_url = "https://api.semanticscholar.org/graph/v1/paper/search"
headers = {
    "Accept": "application/json",
}

# Parameters for the request
params = {
    "query": "",
    "fields": "title,abstract",
    "limit": 100,
}

# List to store all collected papers
all_papers = []

# Loop through each query
for query in queries:
    params["query"] = query
    for i in range(0, 1000, 100):
        params["offset"] = i
        response = requests.get(api_url, headers=headers, params=params)

        if response.status_code == 200:
            papers = response.json().get('data', [])
            all_papers.extend(papers)
            print(f"Collected papers for query: '{query}' with offset {i}.")
        elif response.status_code == 429:
            print(f"Rate limit exceeded. Waiting 120 seconds before retrying for query: '{query}'")
            time.sleep(120)  # Wait 2 minutes before retrying
            response = requests.get(api_url, headers=headers, params=params)
            if response.status_code == 200:
                papers = response.json().get('data', [])
                all_papers.extend(papers)
                print(f"Collected papers after retry for query: '{query}' with offset {i}.")
            else:
                print(f"Error after retrying: {response.status_code}, Query: {query}, Offset: {i}")
                break
        else:
            print(f"Error fetching data: {response.status_code}, Query: {query}, Offset: {i}")
            break

        time.sleep(10)  # Increase time between requests

# Extract titles and abstracts
abstracts = [(paper["title"], paper.get("abstract", "N/A")) for paper in all_papers]

# Save to CSV file
csv_file = "research_abstracts.csv"
df = pd.DataFrame(abstracts, columns=["Title", "Abstract"])
df.to_csv(csv_file, index=False, encoding='utf-8')

print(f"Collected abstracts. Saved to {csv_file}.")

# Open the CSV file
if platform.system() == "Windows":
    os.startfile(csv_file)
elif platform.system() == "Darwin":
    subprocess.call(["open", csv_file])
else:
    subprocess.call(["xdg-open", csv_file])

df.head(15)


Rate limit exceeded. Waiting 120 seconds before retrying for query: 'machine learning'
Error after retrying: 429, Query: machine learning, Offset: 0
Collected papers for query: 'data science' with offset 0.
Rate limit exceeded. Waiting 120 seconds before retrying for query: 'data science'
Error after retrying: 429, Query: data science, Offset: 100
Rate limit exceeded. Waiting 120 seconds before retrying for query: 'artificial intelligence'
Error after retrying: 429, Query: artificial intelligence, Offset: 0
Rate limit exceeded. Waiting 120 seconds before retrying for query: 'information extraction'
Error after retrying: 429, Query: information extraction, Offset: 0
Collected abstracts. Saved to research_abstracts.csv.


,Title,Abstract
0,"Reproducible, interactive, scalable and extens...",None
1,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ..."
2,Spatial Data Science,None
3,The Role of ChatGPT in Data Science: How AI-As...,"ChatGPT, a conversational AI interface that ut..."
4,DS-1000: A Natural and Reliable Benchmark for ...,"We introduce DS-1000, a code generation benchm..."
5,What is Data Science?,"The Communications website, https://cacm.acm.o..."
6,Data Science: 8th International Conference of ...,None
7,The R Language: An Engine for Bioinformatics a...,The R programming language is approaching its ...
8,Smart Health Intelligent Healthcare Systems in...,"In recent decades, healthcare organizations ar..."
9,Training and Evaluating a Jupyter Notebook Dat...,We study the feasibility of a Data Science ass...


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [2]:
import nltk

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import nltk

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')  # Download WordNet resource


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [5]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
import os
import platform

# Load the CSV file
csv_file = "research_abstracts.csv"
df = pd.read_csv(csv_file)

# Initialize necessary components
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Function to clean text data
def clean_text(text):
    # Check if the text is NaN or not a string, return an empty string if true
    if not isinstance(text, str):
        return ""

    # (1) Remove noise, such as special characters and punctuations
    text = re.sub(r'[^\w\s]', '', text)

    # (2) Remove numbers
    text = re.sub(r'\d+', '', text)

    # (3) Remove stopwords
    text = ' '.join(word for word in text.split() if word.lower() not in stop_words)

    # (4) Lowercase all texts
    text = text.lower()

    # (5) Stemming
    text = ' '.join(stemmer.stem(word) for word in text.split())

    # (6) Lemmatization
    text = ' '.join(str(Word(word).lemmatize()) for word in text.split())

    return text

# Apply the clean_text function to the abstracts
df['Cleaned_Abstract'] = df['Abstract'].apply(clean_text)

# Save the cleaned data to a new CSV file
cleaned_csv_file = "cleaned_research_abstracts.csv"
df.to_csv(cleaned_csv_file, index=False, encoding='utf-8')

print(f"Cleaned data saved to {cleaned_csv_file}.")

df.head(15)


Cleaned data saved to cleaned_research_abstracts.csv.


,Title,Abstract,Cleaned_Abstract
0,"Reproducible, interactive, scalable and extens...",NaN,
1,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ...",cambridg univers press let u summar find rando...
2,Spatial Data Science,NaN,
3,The Role of ChatGPT in Data Science: How AI-As...,"ChatGPT, a conversational AI interface that ut...",chatgpt convers ai interfac util natur languag...
4,DS-1000: A Natural and Reliable Benchmark for ...,"We introduce DS-1000, a code generation benchm...",introduc d code gener benchmark thousand data ...
5,What is Data Science?,"The Communications website, https://cacm.acm.o...",commun websit httpscacmacmorg featur dozen blo...
6,Data Science: 8th International Conference of ...,NaN,
7,The R Language: An Engine for Bioinformatics a...,The R programming language is approaching its ...,r program languag approach th birthday last th...
8,Smart Health Intelligent Healthcare Systems in...,"In recent decades, healthcare organizations ar...",recent decad healthcar organ around world incr...
9,Training and Evaluating a Jupyter Notebook Dat...,We study the feasibility of a Data Science ass...,studi feasibl data scienc assist power sequenc...


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [6]:
!pip install nltk spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [12]:
import pandas as pd
import nltk
import spacy
from nltk import pos_tag, word_tokenize
from nltk import ne_chunk
from nltk.tree import Tree

# Load the cleaned CSV file
cleaned_csv_file = "cleaned_research_abstracts.csv"
df = pd.read_csv(cleaned_csv_file.encode('utf-8').decode('utf-8'))

# Initialize spaCy model for NER and parsing
nlp = spacy.load("en_core_web_sm")

# Function to perform POS tagging and count POS types
def pos_analysis(text):
    # Ensure the text is a string
    if not isinstance(text, str):
        return [], {}

    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)

    counts = {
        'Nouns': sum(1 for word, tag in tagged if tag.startswith('NN')),
        'Verbs': sum(1 for word, tag in tagged if tag.startswith('VB')),
        'Adjectives': sum(1 for word, tag in tagged if tag.startswith('JJ')),
        'Adverbs': sum(1 for word, tag in tagged if tag.startswith('RB')),
    }

    return tagged, counts

# Function to perform constituency and dependency parsing
def parsing_analysis(text):
    if not isinstance(text, str):
        return None
    return nlp(text)

# Function to extract named entities and their counts
def ner_analysis(text):
    if not isinstance(text, str):
        return {}
    doc = nlp(text)
    entities = {}

    for ent in doc.ents:
        entities[ent.label_] = entities.get(ent.label_, 0) + 1

    return entities

# Perform analysis for each abstract and store results
results = []

for index, row in df.iterrows():
    abstract = row['Cleaned_Abstract']

    # Skip empty abstracts
    if pd.isna(abstract) or abstract.strip() == "":
        continue

    # POS Analysis
    tagged, pos_counts = pos_analysis(abstract)

    # Parsing Analysis
    parsed_doc = parsing_analysis(abstract)

    # NER Analysis
    entities = ner_analysis(abstract)

    results.append({
        'Abstract': abstract,
        'POS Tagged': tagged,
        'POS Counts': pos_counts,
        'Dependency Parsing': [(sent.text, [(token.text, token.dep_, token.head.text) for token in sent]) for sent in parsed_doc.sents] if parsed_doc else [],
        'Named Entities': entities
    })

# Save the results to a new CSV file
results_df = pd.DataFrame(results)
results_df.to_csv("cleaned_data.csv", index=False)

# Example Output of Parsing Analysis for the first abstract
if results:
    first_abstract = results[0]['Abstract']
    parsed_doc = parsing_analysis(first_abstract)

    # Print the dependency parse tree for the first sentence
    print(f"\nDependency Parsing for the first sentence in the first abstract:")
    for sent in parsed_doc.sents:
        print(f"\nSentence: {sent.text}")
        for token in sent:
            print(f"Token: {token.text}, Dependency: {token.dep_}, Head: {token.head.text}")

    # Print out POS counts for the first abstract
    print(f"\nPOS Counts for the first abstract:")
    print(results[0]['POS Counts'])

    # Print out Named Entities for the first abstract
    print(f"\nNamed Entities for the first abstract:")
    print(results[0]['Named Entities'])
else:
    print("No valid abstracts found for analysis.")



Dependency Parsing for the first sentence in the first abstract:

Sentence: cambridg univers press let u summar find random project set r n onto mdimension subspac approxim preserv geometri
Token: cambridg, Dependency: compound, Head: univers
Token: univers, Dependency: compound, Head: press
Token: press, Dependency: npadvmod, Head: let
Token: let, Dependency: ROOT, Head: let
Token: u, Dependency: compound, Head: summar
Token: summar, Dependency: nsubj, Head: find
Token: find, Dependency: ccomp, Head: let
Token: random, Dependency: amod, Head: project
Token: project, Dependency: dobj, Head: find
Token: set, Dependency: acl, Head: project
Token: r, Dependency: compound, Head: n
Token: n, Dependency: dobj, Head: set
Token: onto, Dependency: prep, Head: set
Token: mdimension, Dependency: compound, Head: subspac
Token: subspac, Dependency: compound, Head: approxim
Token: approxim, Dependency: compound, Head: preserv
Token: preserv, Dependency: pobj, Head: onto
Token: geometri, Dependency:

In [13]:
from google.colab import files
files.download('cleaned_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

In [ ]:
'''
Here is the cleaned data CSV:
https://myunt-my.sharepoint.com/:x:/g/personal/sreyeshvarmakonduru_my_unt_edu/EX4EqHmAgKRHstI_XgUmik0B7XFG4Fxnv4Plr4EdaaZmAw?e=xrKNRN
'''

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
# Write your response below